In [1]:
from typing import List, Optional, Tuple
import numpy as np
from mealymarkov import MarkovMealyModel

# Example small model (n=4 states, V=2 tokens) that satisfies the constraints.
n = 3
V = 2

# We construct T^0 and T^1 so that T^0 + T^1 is row-stochastic (rows sum to 1).
T0 = np.array([
    [0, 1, 0],
    [0, 0, 1],
    [0, 0, 0.5]
])

T1 = np.array([
    [0, 0, 0],
    [0, 0, 0],
    [0.5, 0, 0]
])

model = MarkovMealyModel(n=n, V=V, T_list=[T0, T1])

# By specification the default eta^0 is uniform
print("Initial eta^0 =", model.eta0)

# Generate 12 tokens
tokens, states = model.sample_sequence(max_new_tokens=30, seed=42)

print("Generated tokens:", tokens)
print("States (eta^t) traversed:")
for i, s in enumerate(states):
    print(f"t={i} ->", np.round(s, 4))

Initial eta^0 = [0.33333333 0.33333333 0.33333333]
Generated tokens: [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]
States (eta^t) traversed:
t=0 -> [0.3333 0.3333 0.3333]
t=1 -> [0.  0.4 0.6]
t=2 -> [0. 0. 1.]
t=3 -> [1. 0. 0.]
t=4 -> [0. 1. 0.]
t=5 -> [0. 0. 1.]
t=6 -> [1. 0. 0.]
t=7 -> [0. 1. 0.]
t=8 -> [0. 0. 1.]
t=9 -> [0. 0. 1.]
t=10 -> [0. 0. 1.]
t=11 -> [0. 0. 1.]
t=12 -> [1. 0. 0.]
t=13 -> [0. 1. 0.]
t=14 -> [0. 0. 1.]
t=15 -> [0. 0. 1.]
t=16 -> [0. 0. 1.]
t=17 -> [1. 0. 0.]
t=18 -> [0. 1. 0.]
t=19 -> [0. 0. 1.]
t=20 -> [1. 0. 0.]
t=21 -> [0. 1. 0.]
t=22 -> [0. 0. 1.]
t=23 -> [1. 0. 0.]
t=24 -> [0. 1. 0.]
t=25 -> [0. 0. 1.]
t=26 -> [0. 0. 1.]
t=27 -> [0. 0. 1.]
t=28 -> [0. 0. 1.]
t=29 -> [0. 0. 1.]
t=30 -> [1. 0. 0.]
